In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv('Titanic-Dataset.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [6]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [7]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']), df['Survived'], test_size=0.2, random_state=42)

In [8]:
X_train.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.500,S
733,2,male,23.0,0,0,13.000,S
382,3,male,32.0,0,0,7.925,S


In [9]:
y_train.head()

331    0
733    0
382    0
704    0
813    0
Name: Survived, dtype: int64

In [10]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age         140
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [11]:
si_age = SimpleImputer()
si_embark = SimpleImputer(strategy='most_frequent')

In [12]:
X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_embark = si_embark.fit_transform(X_train[['Embarked']])

In [13]:
X_test_age = si_age.fit_transform(X_test[['Age']])
X_test_embark = si_embark.fit_transform(X_test[['Embarked']])


In [14]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age         140
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [15]:
ohe_sex = OneHotEncoder(sparse=False,handle_unknown='ignore')
ohe_embark = OneHotEncoder(sparse=False,handle_unknown='ignore')

X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_train_embark = ohe_embark.fit_transform(X_train_embark)

X_test_sex = ohe_sex.fit_transform(X_test[['Sex']])
X_test_embark = ohe_embark.fit_transform(X_test_embark)

In [16]:
X_train_sex

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [17]:
X_train_embark

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [18]:
X_train_remain = X_train.drop(columns=['Sex','Age','Embarked'])
X_test_remain = X_test.drop(columns=['Sex','Age','Embarked'])

In [19]:
X_train_trans = np.concatenate((X_train_remain,X_train_age,X_train_sex,X_train_embark),axis = 1)
X_test_trans = np.concatenate((X_test_remain,X_test_age,X_test_sex,X_test_embark),axis = 1)

In [20]:
X_train_trans.shape

(712, 10)

In [21]:
X_test_trans.shape

(179, 10)

In [33]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trans,y_train)

DecisionTreeClassifier()

In [34]:
y_pred = clf.predict(X_test_trans)

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7597765363128491

In [36]:
import pickle

In [39]:
pickle.dump(ohe_sex,open('models/ohe_sex.pkl','wb'))
pickle.dump(ohe_embark,open('models/ohe_embark.pkl','wb'))
pickle.dump(clf,open('models/clf.pkl','wb'))